# Reinforcment Benchmarks

Based on the work of: source: https://www.kaggle.com/code/shantanu199/q-learning

## Init

In [1]:
import gym # openAi gym
# import torch.nn as nn
# import torch
import random
import numpy as np
from IPython.display import clear_output
import time

In [2]:
seed = 123

In [3]:
random.seed(seed)
np.random.seed(seed)
# torch.random.manual_seed(seed)

In [4]:
from simulation.base import Grid

ImportError: cannot import name 'ObsType' from 'gym.core' (C:\Users\lukas\anaconda3\lib\site-packages\gym\core.py)

In [7]:
taxi_env = gym.make('Taxi-v3')

env = taxi_env
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [9]:
# Action Space & Obersvation Space
print(env.action_space.n)
print(env.observation_space.n)

6
500


In [15]:
# Environment Parameters
actions = {0: 'South', 1: 'North', 2: 'East', 3: 'West', 4: 'Pickup', 5: 'Dropoff'}
passenger = {'R': 0, 'G': 1, 'Y': 2, 'B': 3, 'Car': 4}
dest = {'R': 0, 'G': 1, 'Y': 2, 'B': 3}


# Reward: each_step=-1, incorrect_pickup_dropoff=-10, correct_dropoff=20
# Penalty: incorrect_pickup_dropoff=1

# Helper Classes

In [16]:
# Brute Force - only exploration
def brute_force(state):
    env.s = state
    epochs = 0
    penalties = 0
    reward = 0
    frames = [] # for animation
    # initial state
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': '--',
        'reward': '--'
        }
    )
    done = False
    
    while not done:
        # Next action is choosen randomly
        action = env.action_space.sample() # chooses random possible action
        state, reward, done, info = env.step(action)
        if reward == -10:
            penalties += 1
            
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1
    return frames, epochs, penalties

In [17]:
# Visualise frames
def print_frames(frames, time_btw_frames=0.1):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        time.sleep(time_btw_frames)

In [5]:
# Train agent to find path using Reinforcement Learning
def train_agent():
    q_table = np.zeros([env.observation_space.n, env.action_space.n])
    q_table = np.zeros([env.observation_space.n, env.action_space.n])
    # Hyperparameters
    # Learning rate
    alpha = 0.1
    # Discount factor: importance to future rewards
    gamma = 0.6
    # Exploration rate
    epsilon = 0.1

    # For plotting metrics
    all_epochs = []
    all_penalties = []

    for i in range(1, 100001):
        state = env.reset()

        epochs, penalties, reward, = 0, 0, 0
        done = False

        while not done:
            if np.random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values

            next_state, reward, done, info = env.step(action) 

#           Q-Learning
            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value

            if reward == -10:
                penalties += 1

            state = next_state
            epochs += 1

        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")

    print("Training finished.\n")
    return q_table

In [19]:
# Run agent trained using Reinforcemnt learning
def run_agent(state, q_table):
    epochs = 0
    penalties, reward = 0, 0
    env.s = state
    
    frames = [] # for animation
    # initial state
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': '--',
        'reward': '--'
        }
    )
    done = False

    while not done:
        # Next action is choosen from q_table
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        if reward == -10:
            penalties += 1
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1
    return frames, epochs, penalties

# Execution

In [20]:
# Loading state
# (taxi row, taxi column, passenger index, destination index)
# Blue letter indicates pickup location
# Pink Letter indicates dropoff location
state = env.encode(4, 4, passenger['Y'], dest['R']) 
env.s = state

# Uncomment next line to load random state
# env.reset()
state = env.s
print("State:", state)
env.render()

State: 488
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Brute Force

In [21]:
frames, epochs, penalties = brute_force(state)
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 200
Penalties incurred: 66


In [22]:
# Visualise agent
print_frames(frames, time_btw_frames=0.03)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 200
State: 389
Action: 2
Reward: -1


## RL 

In [23]:
%%time
# Train Agent on 100000 episodes
q_table = train_agent()

Episode: 100000
Training finished.

Wall time: 1min 2s


In [24]:
# Printing the best action for our initial state
env.s = state
env.render()
print("Best action: {}".format(actions[np.argmax(q_table[state])]))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Best action: North


In [25]:
# Running agent trained using Reinforcement Learning
frames, epochs, penalties = run_agent(state, q_table)
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 185
Penalties incurred: 0


In [26]:
print_frames(frames,time_btw_frames=0.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

Timestep: 185
State: 410
Action: 0
Reward: -1


# Comparison

In [26]:
%%time
episodes = 100
brute_steps = 0
brute_penalties = 0
reinf_steps = 0
reinf_penalties = 0

for _ in range(episodes):
    env.reset()
    state = env.s
    
    frames, steps, penalties = brute_force(state)
    brute_steps += steps
    brute_penalties += penalties
    
    frames, steps, penalties = run_agent(state, q_table)
    reinf_steps += steps
    reinf_penalties += penalties
    
print('Result after {} episodes'.format(episodes))
print('Average Brute-Force timestamps per episode           : {}'.format(brute_steps/episodes))
print('Average Brute-Force penalties per episode            : {}'.format(brute_penalties/episodes))
print('Average Reinforcement Learning timestamps per episode: {}'.format(reinf_steps/episodes))
print('Average Reinforcement Learning penalties per episode : {}'.format(reinf_penalties/episodes))

Result after 100 episodes
Average Brute-Force timestamps per episode           : 195.07
Average Brute-Force penalties per episode            : 62.65
Average Reinforcement Learning timestamps per episode: 5.85
Average Reinforcement Learning penalties per episode : 0.0
CPU times: user 1.34 s, sys: 31.6 ms, total: 1.37 s
Wall time: 1.3 s
